In [1]:
import bokit as bk

# Decomposition on a complex example

The complex rule was taken from a Boolean model of the cell division cycle in S. cerevisiae: https://www.nature.com/articles/s41467-019-08903-w

We have a function with a large estimated complexity, but for which we can still compute the 570 prime implicants.
Decomposing this functions gives a system of 6 expressions and a total of 54 prime implicants (10 fold reduction).

In [2]:
cplx = "((R43 & R939 & S1166) | ((S45 | S44) & (S43 | S42) & (S47 | S46) & (S431 | S1167) & (S575 | S326 | S1169) & (S1166 | S47) & (S1173 | S1172) & (S1168 | S471) & (S1171 | S1170) & ((R43 & (R939 | S46 | ((S45 | S44) & (S43 | S42) & (S47 | S46) & ((S47 & R44) | (S46 & !((R43 & S1166 & S46)))))) & (S1166 | ((S431 | S1167) & (S575 | S326 | S1169) & (S1166 | S47) & (S1173 | S1172) & (S1168 | S471) & (S1171 | S1170) & ((S47 & R44) | (S1166 & !((R43 & S1166 & S46))))))) | (S47 & !((S47 & R44))))))"

In [3]:
vs = bk.VarSpace()
elx = vs.parse_expression(cplx, extend=True)
len(elx.regulators()), elx.estimate_complexity()

(21, ExprComplexity { score: Some(10054657), atoms: 60, depth: 12 })

In [4]:
(~elx).estimate_complexity()

ExprComplexity { score: Some(507), atoms: 60, depth: 12 }

In [5]:
npr = bk.Primes(~elx)
npr.len()

25

In [6]:
elx.estimate_complexity()

ExprComplexity { score: Some(10054657), atoms: 60, depth: 12 }

In [7]:
%time pr = bk.Primes(elx)

CPU times: user 148 ms, sys: 913 µs, total: 149 ms
Wall time: 149 ms


In [8]:
pr.len()

577

In [9]:
nepr = bk.Expr(npr)
nepr.regulators().len()

21

In [10]:
vs.display(elx)

'R43 & R939 & S1166 | (S45 | S44) & (S43 | S42) & (S47 | S46) & (S431 | S1167) & (S575 | S326 | S1169) & (S1166 | S47) & (S1173 | S1172) & (S1168 | S471) & (S1171 | S1170) & (R43 & (R939 | S46 | (S45 | S44) & (S43 | S42) & (S47 | S46) & (S47 & R44 | S46 & (!R43 | !S1166 | !S46))) & (S1166 | (S431 | S1167) & (S575 | S326 | S1169) & (S1166 | S47) & (S1173 | S1172) & (S1168 | S471) & (S1171 | S1170) & (S47 & R44 | S1166 & (!R43 | !S1166 | !S46))) | S47 & (!S47 | !R44))'

In [11]:
vs.display(nepr)

'!R43 & !S45 & !S44 | !R939 & !S45 & !S44 | !S1166 & !S45 & !S44 | !R43 & !S43 & !S42 | !R939 & !S43 & !S42 | !S1166 & !S43 & !S42 | !S1166 & !S1171 & !S1170 | !R939 & !S1171 & !S1170 | !S1166 & !S575 & !S326 & !S1169 | !R43 & !S431 & !S1167 | !R939 & !S431 & !S1167 | !S1166 & !S431 & !S1167 | !R43 & !S575 & !S326 & !S1169 | !R939 & !S575 & !S326 & !S1169 | !R43 & !S1171 & !S1170 | !R43 & !S1173 & !S1172 | !R939 & !S1173 & !S1172 | !S1166 & !S1173 & !S1172 | !R43 & !S1168 & !S471 | !R939 & !S1168 & !S471 | !S1166 & !S1168 & !S471 | !S1166 & !S47 | !R939 & !S47 & !S46 | !R43 & !S47 | R44 & !R43'

In [12]:
nepr.estimate_complexity()

ExprComplexity { score: Some(25), atoms: 75, depth: 25 }

In [13]:
epr = ~ nepr

In [14]:
epr.estimate_complexity()

ExprComplexity { score: Some(595077871104), atoms: 75, depth: 25 }

In [15]:
# In this case, the prime implicants avoid most of the potential complexity
%time primes = bk.Primes(epr)
primes.len()

CPU times: user 20 ms, sys: 0 ns, total: 20 ms
Wall time: 20.2 ms


577

In [16]:
dc,rep = epr.decompose()
print(rep)
dc

DecomposeReport { root_score: 72, sum_sub_score: 495 }


Expanded __25__ & (__2__ | __16__ | __17__) & (__2__ | __7__) & (__1__ | __7__ | __8__) & (__0__ | __7__) & (!__20__ | __0__)
  _25_ <- __24__ & (__1__ | __14__ | __15__) & (__2__ | __14__ | __15__) & (__0__ | __16__ | __17__) & (__1__ | __16__ | __17__)
  _21_ <- (__0__ | __3__ | __4__) & (__1__ | __3__ | __4__) & (__2__ | __3__ | __4__) & (__0__ | __5__ | __6__)
  _23_ <- __22__ & (__2__ | __11__ | __12__ | __13__) & (__0__ | __9__ | __10__) & (__1__ | __9__ | __10__) & (__2__ | __9__ | __10__)
  _22_ <- __21__ & (__1__ | __5__ | __6__) & (__2__ | __5__ | __6__) & (__2__ | __18__ | __19__) & (__1__ | __18__ | __19__)
  _24_ <- __23__ & (__0__ | __11__ | __12__ | __13__) & (__1__ | __11__ | __12__ | __13__) & (__0__ | __18__ | __19__) & (__0__ | __14__ | __15__)

In [17]:
# we can then get the prime implicants of the decomposed expression
dc_primes = dc.to_primes()
print(f"The decomposed expression has {dc_primes.expansion_count()} associated rules and {dc_primes.total_len()} prime implicants in total")

The decomposed expression has 5 associated rules and 54 prime implicants in total


In [18]:
dc_primes

[[Expanded
Root: 
-------1---------1--0----1
-------1--------1---0----1
--1----1------------0----1
111----------------------1
1-1-----1----------------1
1-1----1-----------------1
1------1--------1--------1
1------1---------1-------1

Associated _21_:
111
1--1
1---1
----11
---1-1
----1-1
---1--1

Associated _22_:
-11------------------1
------1-----------1--1
-----1------------1--1
-----1-------------1-1
------1------------1-1

Associated _24_:
11---------------------1
1-----------1----------1
1----------1-----------1
1------------1---------1
-----------1--1----1---1
------------1-1----1---1
-------------11----1---1
-----------1--1---1----1
-------------11---1----1
------------1-1---1----1
-----------1---1---1---1
------------1--1---1---1
-------------1-1---1---1
-----------1---1--1----1
-------------1-1--1----1
------------1--1--1----1

Associated _23_:
111-------------------1
--1------1------------1
---------1---1--------1
---------1--1---------1
---------1-1----------1
--1-------1---